In [28]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split 
from sklearn import metrics  
import matplotlib.pyplot as plt  
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import plot_tree 
from sklearn.ensemble import RandomForestClassifier

In [2]:
os.chdir(r"File location")

In [3]:
data=pd.read_csv("Churn_Modelling.csv")

In [4]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
data.drop(columns=["RowNumber","CustomerId","Surname"],inplace=True)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [8]:
data.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
data.describe(percentiles=[.01,.05,.1,.2,.25,.5,.75,.90,.95,.99]).T

,count,mean,std,min,1%,5%,10%,20%,25%,50%,75%,90%,95%,99%,max
CreditScore,10000.0,650.528800,96.653299,350.00,432.0000,489.0000,521.00,566.000,584.00,652.000,718.0000,778.000,812.0000,850.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,21.0000,25.0000,27.00,31.000,32.00,37.000,44.0000,53.000,60.0000,72.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,0.0000,1.0000,1.00,2.000,3.00,5.000,7.0000,9.000,9.0000,10.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.0000,0.0000,0.00,0.000,0.00,97198.540,127644.2400,149244.792,162711.6690,185967.9854,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.0000,1.0000,1.00,1.000,1.00,1.000,2.0000,2.000,2.0000,3.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.0000,0.0000,0.00,0.000,0.00,1.000,1.0000,1.000,1.0000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.0000,0.0000,0.00,0.000,0.00,1.000,1.0000,1.000,1.0000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,1842.8253,9851.8185,20273.58,41050.736,51002.11,100193.915,149388.2475,179674.704,190155.3755,198069.7345,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.0000,0.0000,0.00,0.000,0.00,0.000,0.0000,1.000,1.0000,1.0000,1.00


In [10]:
data=pd.get_dummies(data,columns=["Geography","Gender"],drop_first=True)

In [11]:
 for col in data.columns:
        percentiles =data[col].quantile([0.01,0.99]).values
        data[col][data[col]<=percentiles[0]]=percentiles[0]
        data[col][data[col]>=percentiles[1]]=percentiles[1]

C:\Users\AKANSHA\AppData\Local\Temp/ipykernel_752/3229209797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col][data[col]<=percentiles[0]]=percentiles[0]
C:\Users\AKANSHA\AppData\Local\Temp/ipykernel_752/3229209797.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col][data[col]>=percentiles[1]]=percentiles[1]


In [12]:
x=data.drop(columns=["Exited"])
y=data ["Exited"]

In [13]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=99)

In [14]:
# sc= StandardScaler()
# train_x = sc.fit_transform(x_train)
# test_x=sc.transform(x_test)

In [14]:
rf=RandomForestClassifier()

In [15]:
rf.fit(x_train, y_train) 
print("Train Accuracy :", rf.score(x_train,y_train)) 
print("Test Accuracy :", rf.score(x_test,y_test))

Train Accuracy : 1.0
Test Accuracy : 0.8656666666666667


In [17]:
# print("R^2 is train: ", rf.score(x_train, y_train))
# print("R^2 is test: ", rf.score(x_test, y_test)) 
# print("MAE of Train: ", np.mean(np.abs((y_train-rf.predict(x_train))) )) 
# print("MAE of Test: ", np.mean(np.abs((y_test-rf.predict(x_test))) )) 
# print("RMSE of Train: ", np.sqrt(np.mean((y_train-rf.predict(x_train))**2) )) 
# print("RMSE of Test: ", np.sqrt(np.mean((y_test-rf.predict(x_test))**2) ))

In [16]:
rf1= RandomForestClassifier(random_state=40) 

param_dist = {'max_depth':[5,6,7,10],
              'min_samples_split':[2,4,6,10],
              'n_estimators':[10,25, 50, 100],
#               'bootstrap': [True, False],
#               'max_features': ['auto', 'log2', None],
              'criterion': ['gini', 'entropy']} 


cv_rf = GridSearchCV(rf1, cv=5,
                     param_grid=param_dist,
                     n_jobs = -1, verbose=1)  


cv_rf.fit(x_train, y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=40), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 10],
                         'min_samples_split': [2, 4, 6, 10],
                         'n_estimators': [10, 25, 50, 100]},
             verbose=1)

In [17]:
 cv_rf.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_split': 10,
 'n_estimators': 25}

In [19]:
rf2= RandomForestClassifier(n_estimators = 25,
                            max_depth = 10,
                            min_samples_split=10,
                            criterion='gini') 
    
rf2.fit(x_train,y_train)

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=25)

In [20]:
rf2.fit(x_train, y_train) 
print("Train Accuracy :", rf2.score(x_train,y_train)) 
print("Test Accuracy :", rf2.score(x_test,y_test))

Train Accuracy : 0.8924285714285715
Test Accuracy : 0.8673333333333333


In [22]:
pred_train_rf=rf2.predict(x_train) 
pred_test_rf=rf2.predict(x_test)

In [25]:
 print(metrics.classification_report(y_train, pred_train_rf))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5595
           1       0.90      0.52      0.66      1405

    accuracy                           0.89      7000
   macro avg       0.90      0.75      0.80      7000
weighted avg       0.89      0.89      0.88      7000



In [27]:
 rf2.predict_proba(x_train)[:,1]

array([0.2727973 , 0.05843571, 0.1234448 , ..., 0.18982929, 0.65901276,
       0.04599957])